In [49]:
import pandas as pd
import numpy as np
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [50]:
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])
    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [51]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [52]:
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [53]:
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header= None)
def load_signals(subset):
    signals_data=[]
    for signal in SIGNALS:
        filename = f'{signal}_{subset}.txt'
        signals_data.append(_read_csv(filename).as_matrix())
        return np.transpose(signals_data, (1, 2, 0))

In [54]:
def load_y(subset):
    filename = f'y_{subset}.txt'
    y = _read_csv(filename)[0]
    return pd.get_dummies(y).as_matrix()

In [55]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [56]:
session_conf=tf.ConfigProto(inter_op_parallelism_threads=1, intra_op_parallelism_threads=1)

In [57]:
from keras import backend as K

In [58]:
sess=tf.Session(graph=tf.get_default_graph(),config=session_conf)
K.set_session(sess)

In [59]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [60]:
epochs = 30
batch_size = 16
n_hidden = 32

In [61]:
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [62]:
X_train, X_test, Y_train, Y_test = load_data()

C:\Users\mondeswa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
C:\Users\mondeswa\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [63]:
print(X_train)

[[[ 1.808515e-04]
  [ 1.013856e-02]
  [ 9.275574e-03]
  ...
  [-1.147484e-03]
  [-2.222655e-04]
  [ 1.575500e-03]]

 [[ 1.093752e-03]
  [ 4.550077e-03]
  [ 2.879173e-03]
  ...
  [-4.646144e-03]
  [-2.941333e-03]
  [-1.598751e-03]]

 [[ 3.531266e-03]
  [ 2.285063e-03]
  [-4.197538e-04]
  ...
  [ 1.245999e-03]
  [ 3.116783e-03]
  [ 2.177658e-03]]

 ...

 [[-2.267175e-01]
  [-1.778169e-01]
  [-1.508691e-01]
  ...
  [-2.949353e-01]
  [-2.434030e-01]
  [-2.152148e-01]]

 [[-6.480597e-02]
  [-7.993300e-02]
  [-6.834966e-02]
  ...
  [-3.150159e-01]
  [-3.307374e-01]
  [-2.723475e-01]]

 [[-1.944707e-01]
  [-1.738025e-01]
  [-1.275446e-01]
  ...
  [-7.190685e-02]
  [-1.422088e-01]
  [-1.709989e-01]]]


In [64]:
print(X_train[0])

[[ 1.808515e-04]
 [ 1.013856e-02]
 [ 9.275574e-03]
 [ 5.065897e-03]
 [ 1.081025e-02]
 [ 4.045101e-03]
 [ 4.757375e-03]
 [ 6.213647e-03]
 [ 3.306744e-03]
 [ 7.571941e-03]
 [ 5.407117e-03]
 [ 6.221276e-03]
 [ 6.895028e-03]
 [ 4.610260e-03]
 [ 7.331333e-03]
 [ 5.077667e-03]
 [ 5.762556e-03]
 [ 6.692748e-03]
 [ 5.443238e-03]
 [ 8.240952e-03]
 [ 6.506451e-03]
 [ 6.531523e-03]
 [ 7.422441e-03]
 [ 5.771769e-03]
 [ 6.240187e-03]
 [ 4.622982e-03]
 [ 4.832962e-03]
 [ 5.498746e-03]
 [ 4.341186e-03]
 [ 5.098018e-03]
 [ 4.268742e-03]
 [ 3.045187e-03]
 [ 3.204192e-03]
 [ 4.520400e-03]
 [ 5.127211e-03]
 [ 3.549791e-03]
 [ 4.234233e-03]
 [ 4.468670e-03]
 [ 3.572702e-03]
 [ 5.136278e-03]
 [ 5.379786e-03]
 [ 4.408520e-03]
 [ 2.244039e-03]
 [-1.183598e-03]
 [-6.795342e-04]
 [ 2.281389e-03]
 [ 3.653728e-03]
 [ 2.312639e-03]
 [-4.042194e-05]
 [ 5.342254e-04]
 [ 2.963263e-03]
 [ 4.399468e-03]
 [ 2.702339e-03]
 [ 3.452310e-04]
 [ 2.916911e-03]
 [ 4.978556e-03]
 [ 3.446870e-03]
 [ 4.378151e-03]
 [ 5.006170e-0

In [67]:
print(X_train[0][0])
print(X_test[0][0])

[0.00018085]
[0.01165315]


In [66]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
1
7352


In [68]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                4352      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 4,550
Trainable params: 4,550
Non-trainable params: 0
_________________________________________________________________


In [69]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [70]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.cast instead.
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 44s 6ms/step - loss: 1.4138 - accuracy: 0.3058 - val_loss: 1.1496 - val_accuracy: 0.3461
Epoch 2/30
7352/7352 [==============================] - 44s 6ms/step - loss: 1.1710 - accuracy: 0.3444 - val_loss: 1.1414 - val_accuracy: 0.3488
Epoch 3/30
7352/7352 [==============================] - 43s 6ms/step - loss: 1.2053 - accuracy: 0.3395 - val_loss: 1.4881 - val_accuracy: 0.2782
Epoch 4/30
7352/7352 [==============================] - 42s 6ms/step - loss: 1.1941 - accuracy: 0.3383 - val_loss: 1.1198 - val_accuracy: 0.3488
Epoch 5/30
7352/7352 [==============================] - 42s 6ms/step - loss: 1.1467 - accuracy: 0.3485 - val_loss: 1.1192 - val_accuracy: 0.3488
Epoch 6/30
7352/7352 [==============================] - 42s 6ms/step - loss: 1.1349 - accuracy: 0.3561 - val_loss: 1.1176 - val_accuracy: 0.3471
Epoch 7/30
7352/7352 [============

In [72]:
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                                                     
LAYING                 536        1                   0                 0
SITTING                490        1                   0                 0
STANDING               532        0                   0                 0
WALKING                  0      474                   2                20
WALKING_DOWNSTAIRS       0        2                 385                33
WALKING_UPSTAIRS         0       47                  92               332


In [73]:
score=model.evaluate(X_test,Y_test)

2947/2947 [==============================] - 2s 699us/step


In [74]:
score

[0.7828343284910801, 0.5860196948051453]